Data prepare 

In [1]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math
from IPython.display import clear_output

c:\users\ntu\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ntu\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ntu\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ntu\anaconda3\envs\tensorflow-gpu\lib\site-

In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
user_following = np.load('../Data/npy/user_following_1489.npy')
all_3374 = np.load('../Data/npy/Image_50_2048D.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')
print('Feature shape:',all_3374.shape)

Feature shape: (88, 2048)


In [4]:
user_test_amount = 150
yt_test_amount = 18

In [5]:
def generate_train_test(user_following,feature,active_users,user_test_amount,yt_test_amount):
    print('Generate_train_test----')
    #The shape of orignal data
    print('user_following shape ',user_following.shape)
    print('feature shape ',feature.shape)
    
    #following youtuber for each user
    following_true = []
    for i in range(len(user_following)):
        each_user = []
        for j in range(len(user_following[i])):
            if user_following[i][j] == 1:
                each_user.append(j)
        following_true.append(each_user)
    #print(following_true)
    #number of followings for each user
    minlen = 10000
    maxlen = 0
    num_of_follower = []
    for i in range(len(following_true)):
        if len(following_true[i]) < minlen:
            minlen = len(following_true[i])
        if len(following_true[i]) > maxlen:
            maxlen = len(following_true[i])
        num_of_follower.append(len(following_true[i]))
    print('Min number of followings ',minlen)
    print('Max number of followings ',maxlen)
    
    num_of_users = user_following.shape[0]
    num_of_youtubers = user_following[1]
    
    """
    Spliting training and testing data
    """
    #testing user id random choice
    user_idx = [i for i in range(len(user_following))]
    random.seed(5)
    #choose test_id and sorted from small to large
    test_idx = sorted(random.sample(user_idx,user_test_amount))
    
    train_t = []
    train_f = []
    test_t = []
    test_f = []
    
    print('test_idx',test_idx)
    test_pos = -1
    for i in range(num_of_users):
        t_for_train = []
        f_for_train = []
        if i not in test_idx: #if not in test id, just append it to true or false list
            for j in range(88):
                if user_following[i][j] == 1:
                    t_for_train.append(j)
                else:
                    f_for_train.append(j)
            train_t.append(t_for_train)
            train_f.append(f_for_train)

        else: #if in test id, choose 2 true and other 
            test_pos += 1
            temp_t = []
            temp_f = []
            for j in range(88):
                if user_following[i][j] == 1:
                    temp_t.append(j)
                else:
                    temp_f.append(j)
            t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
            f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))

            test_t.append(t_for_test)
            test_f.append(f_for_test)

            #other for training
            t_for_train = [item for item in temp_t if not item in t_for_test]
            f_for_train = [item for item in temp_f if not item in f_for_test]
            train_t.append(t_for_train)
            train_f.append(f_for_train)
            
    # train_t[i] 代表的是user i positive feedback
    print('The length of train_t:',len(train_t))
    print('The length of train_f:',len(train_f))
    print('The length of test_t:',len(test_t))
    print('The length of test_f:',len(test_f))
    return train_t,train_f,test_t,test_f

In [6]:
train_t,train_f,test_t,test_f = generate_train_test(user_following,all_3374,active_users,user_test_amount,yt_test_amount)

Generate_train_test----
user_following shape  (1489, 88)
feature shape  (88, 2048)
Min number of followings  5
Max number of followings  34
test_idx [3, 6, 10, 18, 26, 37, 44, 46, 59, 65, 67, 75, 95, 99, 106, 114, 116, 133, 135, 147, 160, 165, 186, 188, 208, 221, 231, 243, 259, 270, 284, 298, 303, 304, 321, 326, 330, 339, 340, 360, 363, 370, 372, 378, 402, 403, 407, 419, 426, 428, 441, 443, 486, 503, 504, 510, 512, 513, 523, 524, 540, 564, 572, 592, 605, 611, 617, 626, 627, 633, 634, 639, 642, 646, 648, 679, 692, 696, 697, 704, 705, 726, 727, 732, 734, 739, 742, 749, 752, 761, 770, 779, 784, 797, 827, 831, 835, 849, 857, 863, 886, 911, 927, 933, 946, 947, 953, 960, 967, 984, 985, 990, 1049, 1050, 1074, 1085, 1092, 1104, 1113, 1116, 1124, 1175, 1184, 1200, 1207, 1216, 1220, 1230, 1235, 1250, 1264, 1265, 1275, 1277, 1281, 1283, 1307, 1329, 1333, 1335, 1388, 1404, 1411, 1414, 1426, 1438, 1443, 1449, 1474, 1476]
The length of train_t: 1489
The length of train_f: 1489
The length of test_t: 

In [7]:
#average num of following for training user
total_train = 0
for t in train_t:
    total_train += len(t)
avg = total_train/len(user_following)
print('training',avg)
#average num of following for testing user
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/user_test_amount
print('testing',avg)

training 9.758226997985226
testing 5.466666666666667


Training 

In [8]:
all_auxilary = [i for i in range(88)]

In [9]:
n = len(user_following)
m = 88  
k = 64
l = all_3374.shape[1]

In [10]:
def training(save_name): 
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    loss_acc_list = []
    t0=time.time()
    
    train_yes_id=[] 
    for q in range(7):
        print('Iteraction:',q)
        train_auc=0
        total_loss=0
        xuij_auc=0
        length = 0
        for z in range(n):
            """
            yes 用來存放選擇到的YouTuber feature (for auxilary)
            """
            yes=[]
            yesr=[]
        
            """
            only choose positive 
            """
            #sample=random.sample(train_t[z],len(train_t[z])) #選全部的Positive
            """
            choose all YouTuber 
            """
            sample= all_auxilary #選全部的Positive
        
            #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))
        
            
         
            for b in range(len(sample)):
                yes.append(all_3374[sample[b]])
        
            yes=np.array(yes)
        
            #取positive 
            train_t_sample = random.sample(train_t[z],len(train_t[z]))
            #print('number of positive feedback', len(train_t_sample))
        
            #train_f_sample = random.sample(train_f[z],20)
            for ta in train_t_sample:
                pos = sample.index(ta)
                
                image_1=np.expand_dims(all_3374[ta],0) #(1,2048)
                train_f_sample = random.sample(train_f[z],10)
                
                for b in train_f_sample:
                    image_2=np.expand_dims(all_3374[b],0) #(1,2048)
                    _a_list,r3,_auc, _loss,_=sess.run([a_list_smooth,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], xf: yes , l_id:sample, l_id_len:[len(sample)],positive_id:train_t[z],positive_len:[len(train_t[z])],
                                        image_i:image_1,image_j:image_2})
                    #print(_a_list)
                    #print(r3)
                    train_auc+=_auc
                    total_loss+=_loss
                    length += 1
        #print('a_list:',_a_list)
        #print('a_list_soft:',r3)
        print("total_loss:-----------------", total_loss/length)
        print("train_auc:-------------------", train_auc/length)
        loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
        print('time:',time.time()-t0,' sec')
    print('Total cost ',time.time()-t0,' sec')   
    U, Y, A,E,Au, Ay, Aa, Av =sess.run([user_latent, item_latent, aux_item, embedding,Wu, Wy, Wa, Wv])
    np.savez('../Data/grid_search_weight/acf/'+save_name+'.npz', 
                        U=U, Y=Y, A=A,E=E,Wu=Au, Wy=Ay, Wa=Aa, Wv=Av)
    return U, Y, A, E, Au, Ay, Aa, Av

In [ ]:
par_weights = [0.01]
beta_weights = [0.001]
Embedding_weights = [0.01]
Embedding_dims = [200]

count_try = ['Img'+str(i) for i in range(10)]

testcount = 0
finish_list = []
for try_i in count_try:
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                for embedding_dims in Embedding_dims:
                    #clear_output()
                    #print('Finished Dims',finish_list)
                    finish_list.append(embedding_dims)
                    #print('Now Dims:',embedding_dims)
                    print(try_i)
                    """
                    n: the number of users
                    m: the number of YouTubers
                    k: latent dims
                    l: feature dims
                    """
                    tf.reset_default_graph()

                    user = tf.placeholder(tf.int32,shape=(1,))
                    i = tf.placeholder(tf.int32, shape=(1,))
                    j = tf.placeholder(tf.int32, shape=(1,))

                    #多少個auxliary 
                    xf = tf.placeholder(tf.float32, shape=(None,l))
                    l_id = tf.placeholder(tf.int32, shape=(None,))
                    l_id_len = tf.placeholder(tf.int32,shape=(1,))
                    positive_id = tf.placeholder(tf.int32, shape=(None,))
                    positive_len = tf.placeholder(tf.int32,shape=(1,))


                    image_i = tf.placeholder(tf.float32, shape=(1,l))
                    image_j = tf.placeholder(tf.float32, shape=(1,l))

                    with tf.variable_scope("item_level"):
                        user_latent = tf.get_variable("user_latent", [n, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        item_latent = tf.get_variable("item_latent", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
                        aux_item = tf.get_variable("aux_item", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        Wu = tf.get_variable("Wu", [n,m,k],  
                                                              initializer=tf.contrib.layers.xavier_initializer())
                        Wy = tf.get_variable("Wy", [n,m,k],   
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wa = tf.get_variable("Wa", [n,m,k],  
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wv = tf.get_variable("Wv", [n,m,embedding_dims],  
                                                            initializer=tf.contrib.layers.xavier_initializer())
                        #Wve =  tf.get_variable("Wve", [embedding_dims,l],  
                        #                                    initializer=tf.contrib.layers.xavier_initializer())

                        with tf.variable_scope('feature_level'):
                            embedding = tf.get_variable("embedding", [embedding_dims,l],
                                        initializer=tf.contrib.layers.xavier_initializer())

                        aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))



                    #lookup the latent factors by user and id
                    u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                    vi = tf.nn.embedding_lookup(item_latent, i) 
                    vj = tf.nn.embedding_lookup(item_latent, j)


                    wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                    wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                    wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)
                    wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(m,l)


                    a_list=tf.Variable([])
                    q = tf.constant(0)
                    def att_cond(q,a_list):
                        return tf.less(q,l_id_len[0])
                    def att_body(q,a_list):
                        xfi = tf.expand_dims(xf[q],0) #(1,l)
                        wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                                tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                tf.matmul(wvui,tf.matmul(embedding,xfi, transpose_b=True)))[0][0])]],0)
                        q += 1
                        return q,  a_list

                    _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                    # for while for smoothing
                    #a_list_soft=tf.nn.softmax(a_list)
                    a_list_smooth = tf.add(a_list,0.0000000001)
                    a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

                    aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
                    q = tf.constant(0)
                    def sum_att_cond(q,aux_np):
                        return tf.less(q,l_id_len[0])

                    def sum_att_body(q,aux_np):
                        #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                        aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                        q += 1
                        return q, aux_np

                    _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                    """
                    for q in range(3): #取q個auxliary item
                        aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    """

                    #tf.print('aux attention:',aux_np)
                    aux_np+=u #user_latent factor + sum (alpha*auxilary)
                    aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                    #矩陣中對應函數各自相乘
                    # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                    xui = tf.matmul(aux_new, vi, transpose_b=True)
                    xuj = tf.matmul(aux_new, vj, transpose_b=True)

                    xuij = tf.subtract(xui,xuj)

                    l2_norm = tf.add_n([
                                0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                                0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                                0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),


                                0.01 * tf.reduce_sum(tf.multiply(wu, wu)),
                                pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                                pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                                pary_weight * tf.reduce_sum(tf.multiply(wv,wv)),
                                Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),
                                ])

                    loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                    train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                    auc = tf.reduce_mean(tf.to_float(xuij > 0))

                    Ur, Yr, Ar, Er, Aur, Ayr, Aar, Avr = training('ACF_NR'+try_i+'_Edims200')
                    print('Finish dims:,',embedding_dims)

Img0
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Iteraction: 0
total_loss:----------------- [[0.66763587]]
train_auc:------------------- 0.6543840330350998
time: 3393.9468097686768  sec
Iteraction: 1
total_loss:----------------- [[0.59380888]]
train_auc:------------------- 0.8120715760495526
time: 6802.922408342361  sec
Iteraction: 2
total_loss:----------------- [[0.51742257]]
train_auc:------------------- 0.8520578114246387
time: 10211.480913162231  sec
Iteraction: 3
total_loss:----------------- [[0.45140189]]
train_auc:------------------- 0.8585409497591191
time: 13616.095241069794  sec
Iteraction: 4
total_loss:----------------- [

# Testing and Evaluation

In [22]:
import os 
import numpy as np
import random
import math
from sklearn.metrics import average_precision_score
import pandas as pd

In [23]:
def square_sum(pars):
    all_squ_sum = []
    for par in pars:
        all_squ_sum.append(np.sum(np.multiply(par,par)))
    return all_squ_sum 
def shape_sum(pars):
    all_shape = []
    for par in pars:
        all_shape.append(par.shape)
    return all_shape
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))  

In [24]:
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)
test_amount = 150
yt_test_amount = 18
user_idx = [i for i in range(len(user_following))]
#user_idx = user_idx_over10
#test_idx is the number of user for testing
random.seed(5)
test_idx = sorted(random.sample(user_idx,test_amount))
print(test_idx)
# Training  and Testing --New
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        #print(len(temp_t),math.ceil(0.5*len(temp_t)))
        t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        #print(len(t_for_train ))
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/test_amount
print(avg)

Min number of followings  5
Max number of followings  34
[3, 6, 10, 18, 26, 37, 44, 46, 59, 65, 67, 75, 95, 99, 106, 114, 116, 133, 135, 147, 160, 165, 186, 188, 208, 221, 231, 243, 259, 270, 284, 298, 303, 304, 321, 326, 330, 339, 340, 360, 363, 370, 372, 378, 402, 403, 407, 419, 426, 428, 441, 443, 486, 503, 504, 510, 512, 513, 523, 524, 540, 564, 572, 592, 605, 611, 617, 626, 627, 633, 634, 639, 642, 646, 648, 679, 692, 696, 697, 704, 705, 726, 727, 732, 734, 739, 742, 749, 752, 761, 770, 779, 784, 797, 827, 831, 835, 849, 857, 863, 886, 911, 927, 933, 946, 947, 953, 960, 967, 984, 985, 990, 1049, 1050, 1074, 1085, 1092, 1104, 1113, 1116, 1124, 1175, 1184, 1200, 1207, 1216, 1220, 1230, 1235, 1250, 1264, 1265, 1275, 1277, 1281, 1283, 1307, 1329, 1333, 1335, 1388, 1404, 1411, 1414, 1426, 1438, 1443, 1449, 1474, 1476]
5.466666666666667


In [25]:
def getScoreMatrix(RS):
    #取出test的資料
    testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*18
    target = np.zeros((test_amount,yt_test_amount)) #(150,18)
    #test_t 是true的
    #test_f 是false的

    for z in range(test_amount):
        user_id = test_idx[z]
        #positive target YouTuber list
        youtube_t = test_t[z] 
        #not target YouTuber list
        youtube_f = test_f[z]

        #前兩個放target的RS
        for i in range(len(youtube_t)):
            testRS[z][i] = RS[z][youtube_t[i]]
            target[z][i] = user_following[user_id][youtube_t[i]]
        for i in range(len(youtube_f)):
            testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]
            target[z][i+len(youtube_t)] = user_following[user_id][youtube_f[i]]
    sumtarget = 0
    for i in range(len(target)):
        #print(np.sum(target[i]))
        sumtarget += np.sum(target[i])
    print('num of positive data in testing:',sumtarget)
    print('total testing data:',test_amount*yt_test_amount)
    return target, testRS,sumtarget

def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [26]:
def getTOP1(target,testRS,sumtarget):
    print('top1')
    correct = 0
    for i in range(len(testRS)):
        top_0 = topN(list(testRS[i]),1) #取一個
        print('top',top_0)
        print('testRS[i]',testRS[i])
        #print(top_0)
        if top_0[0] < int(np.sum(target[i])):
            correct += 1
    top1_prec = correct/len(testRS)
    top1_recall = correct/(sumtarget)
    print('prec ',top1_prec,'recall ',top1_recall)
    #f1 score
    print('F1_score:',F1_score(top1_prec,top1_recall))
    return top1_prec,top1_recall,F1_score(top1_prec,top1_recall)
def getTOP3(target,testRS,sumtarget):
    print('top3')
    correct = 0
    for i in range(len(testRS)):
        top_3 = topN(list(testRS[i]),3) #取一個
        
        #print(top_3)
        for j in range(len(top_3)):
            if top_3[j] < int(np.sum(target[i])):
                correct += 1
    top3_prec = correct/(len(testRS)*3)
    top3_recall = correct/(sumtarget)
    print('prec ',top3_prec,'recall ',top3_recall)
    #f1 score
    print('F1_score:',F1_score(top3_prec,top3_recall))
    return top3_prec,top3_recall,F1_score(top3_prec,top3_recall)
def getTOP5(target,testRS,sumtarget):
    print('top5')
    correct = 0
    for i in range(len(testRS)):
        top_5 = topN(list(testRS[i]),5) #取一個
        print('top',top_5)
        print('testRS[i]',testRS[i])
        #print(top_5)
        for j in range(len(top_5)):
            if top_5[j] < int(np.sum(target[i])):
                correct += 1
    top5_prec = correct/(len(testRS)*5)
    top5_recall = correct/(sumtarget)
    print('prec ',top5_prec,'recall ',top5_recall)
    #f1 score
    print('F1_score:',F1_score(top5_prec,top5_recall))
    return top5_prec,top5_recall,F1_score(top5_prec,top5_recall)

In [31]:
"""
NDCG
https://daiwk.github.io/posts/nlp-ndcg.html
"""
# pre_list
"""
test_amount = 150
yt_test_amount = 18
"""
def NDCG(target,testRS): #target是真正的喜好
    all_sort = []
    num_ndcg = 10
    pre_matrix = np.zeros(shape=(test_amount,yt_test_amount)) #(150,18)
    for i in range(test_amount): #user amount = 150
        top_n = topN(list(testRS[i]),num_ndcg) #取10個
        #print(top_n)
        all_sort.append(topN(list(testRS[i]),len(testRS[i])))
        #print('all_sort',topN(list(testRS[i]),len(testRS[i])))
        for j in range(len(top_n)):
            pre_matrix[i][top_n[j]] = 1

    #Ideal DCG，理想状况下的DCG。也就是说，相关性完全由高到低排序时算出的DCG：
    def IDCG(ideal_list): #ideal_list example = [1,1,1,1,1,0,0,....]
        idcg=0
        print('ideal',ideal_list)
        for i in range(len(ideal_list)):
            #print((2**true_list[i]-1),math.log2(i+2))
            idcg+= (2**ideal_list[i]-1)/ math.log2(i+2)
        #print('idcg',idcg)
        return idcg
    def DCG(prec_list): #找出前n名的[1,1,1,0,...]
        dcg=0
        print('prec',prec_list)
        for i in range(len(prec_list)):
            dcg+= (2**prec_list[i]-1)/ math.log2(i+2)
        #print('dcg',dcg)
        return dcg
    total_ndcg = 0
    
    for m in range(test_amount): # the number of testing users
        idcg = IDCG(target[m][:num_ndcg])
        pre_list = []
        least_pre_list = []
        for s in all_sort[m][:num_ndcg]:
            #print(m,s,target[m][s])
            pre_list.append(target[m][s]) #把prec_list 的 score加進去
        for s in all_sort[m][num_ndcg:]:
            #print(s)
            #print(target[m][s])
            least_pre_list.append(target[m][s]) #把prec_list 的 score加進去
        dcg = DCG(pre_list)
        ndcg = dcg/idcg
        print(ndcg)
        total_ndcg += ndcg
    avg_ndcg = total_ndcg/test_amount
    print('NDCG:',avg_ndcg)
    return pre_matrix,avg_ndcg

# MAP
"""
>>> y_true = np.array([0, 0, 1, 1])
>>> y_scores = np.array([0.1, 0.4, 0.35, 0.8])
>>> average_precision_score(y_true, y_scores)
"""
def MAP(target,testRS):
    print('target:',target)
    print('predict:',testRS)
    total_prec = 0
    for u in range(test_amount):
        y_true = target[u]
        y_scores = testRS[u]
        total_prec+=average_precision_score(y_true, y_scores)
    Map_value = total_prec/test_amount
    print('MAP',Map_value)
    return Map_value

In [32]:
def testing(U, Y, A, E,Au, Ay, Aa, Av):

    test_amount = 150
    yt_test_amount = 18
    result=np.zeros((test_amount,88))
    RS=np.zeros((test_amount,88))
    #test_idx --> Test 的 index
    sum_alpha = 0
    for s in range(test_amount):
        #print(s,test_idx[s])

        yes=[]
        #sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        sample = train_t[test_idx[s]]
        #sample=result_yes_id[now]
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            alpha_a = np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]][sample[a]],
                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]][sample[a]],np.dot(E,np.expand_dims(all_3374[sample[a]],0).T))
            """
            relu part ...
            """
            alpha[a]=np.sum((relu(alpha_a)))
            """
            tanh part ...
            """
            #alpha[a]=np.sum((np.tanh(alpha_a)))*r
            
            
        mul=np.zeros((1,64))
        #print('alpha--------',alpha)
        #print('add alpha------------',np.add(alpha,0.000000001))
        added_alpha = np.add(alpha,0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        #print('alpha-----------',alpha)
        #print('norm alpha--------------',norm_alpha)
        sum_alpha += np.sum(alpha)
        for i in range(len(sample)):
            mul+=norm_alpha[i]*A[sample[i]] #attention alpha*Ai part
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(88):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)
            
    print('sum_alpha',sum_alpha)
    return RS

In [33]:
#With Embedding
import os 
#從grid_search_weight中找尋不同的file 
path = '../Data/grid_search_weight/acf/'
all_files = os.listdir(path)
new_files = []
for file in all_files:
    if 'Edims200.npz' in file:
        new_files.append(file)
all_files = new_files
all_files = ['ACF_NRImg0_Edims200.npz']

In [34]:
par_files = list(set([file.split('_Img')[0] for file in all_files]))
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = '../Data/grid_search_weight/acf/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv)
            target,testRS,sumtarget = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,testRS)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('_Edims')[0])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['ACF_NRImg0_Edims200.npz']
ACF_NRImg0_Edims200.npz
sum_alpha 0.37302859869672034
num of positive data in testing: 820.0
total testing data: 2700
top1
top [0]
testRS[i] [ 2.13418364  0.38409903  0.95491508  1.15919984  0.77029309 -1.38213524
 -1.86044915 -1.41015237  0.06438586 -1.17409697 -0.7605142  -1.89255878
 -1.64145802 -0.10401051 -0.54132622  0.46069338 -1.8014912  -1.76900365]
top [9]
testRS[i] [ 1.24286984  1.73465105  2.04605741 -1.41222311 -0.27595298 -2.10017875
 -2.32843258 -2.4413342  -1.87146824  2.35428083 -2.66158385 -0.80233363
  0.62527608 -1.1104148  -2.52957727  0.05312671  0.31089852 -2.09096478]
top [2]
testRS[i] [-0.83210434  1.06599329  1.09801671  0.84495278  0.62164489  0.68521865
 -1.33058944 -1.61905705 -0.24898099 -1.54088874 -0.46912359 -1.46348513
  0.39146648 -0.65637157 -1.32590385 -1.42081049  0.06930859 -1.62715734]
top [0]
testRS[i] [ 1.54791223  0.48248042  0.71877086  0.70887429  0.50797854 -0.55183236
 -1.27521165 -1.51635953 -0.04457434 -1.5223

0.8704291393287218
ideal [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.7365896932159578
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]
0.971476156593264
ideal [1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
0.783582028747806
ideal [1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
1.0
ideal [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
prec [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]
0.7827392871477702
ideal [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
0.9930783166417602
ideal [1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
prec [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0]
0.9643632283499571
ideal [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
prec [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.20210734650054757
ideal [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]

PermissionError: [Errno 13] Permission denied: '../Data/grid_search_weight/acf/ACF_NRImg0_Edims200.npz.csv'